In [1]:
from moist_konrad import *
import matplotlib.pyplot as plt
import numpy as np

In [2]:
results_file = open("results_file_1.txt",'w')

In [3]:
cases = [1]
initial_temperatures = [285.]
land_atm_coupling = True
strong_coupling = True

In [4]:
%%time
for k in range(len(cases)):
    for te in range(len(initial_temperatures)):
        ###PARAMETERS OF THE TEST:

        #flux for efficiency computation:
        # 1: LH + SH
        # 2: Radiation surface
        # 3: Atmosphere radiation
        case = cases[k]


        #coupled surface-atmosphere: 
        #if true, surface fluxes and surface temperature change at each step, constant otherwise
        coupled = land_atm_coupling

        SH_f = 15.
        LH_f = 140.
        Ts_f = np.array([initial_temperatures[te]])
        print(Ts_f)

        RLS = 1/1

        #Strong coupling surface-atmosphere:
        #If true the atmosphere moist adiabat starts at surface temperature
        #If false find the atmopsheric moist adiabat that conserves energy
        strong = strong_coupling

        popke = False
        if popke == True:
            solar = 510.
            angle = 47.9
        else:
            solar = 480.
            angle = 42.05

        A = 0.2

        ### INITIALIZATION
        E_imbalance = 0.

        plev, phlev = konrad.utils.get_pressure_grids(1000e2, 100, 128)
        atmosphere = konrad.atmosphere.Atmosphere(phlev)
        clearsky = konrad.cloud.ClearSky.from_atmosphere(atmosphere)
        surface = konrad.surface.SlabOcean(
            temperature=288,
            heat_sink=0,
            depth = 50,
            albedo = 0.5)

        radiation = konrad.radiation.RRTMG(solar_constant=solar, zenith_angle = angle)

        ndays = 10000
        timestep = 0.5 #days
        nsteps = int(ndays/timestep)

        printcounter = 0
        status = 0

        tol = 1e-5

        ### RCPE COMPUTATION

        surface['temperature'] = Ts_f
        surface.albedo = A
        atmosphere['T'][0] = moist_adiabat(surface['temperature'],atmosphere['T'][0],atmosphere)
        print(surface['temperature'],surface.albedo)

        conv_top = coldpoint(atmosphere['T'][0])
        RH = manabe_rh(0.77,atmosphere['plev'])
        atmosphere['H2O'][0] = rh_to_vmr(RH,atmosphere['T'][0],atmosphere['plev'],conv_top)
        T_atm_low = surface['temperature']

        print(surface['temperature'],surface.albedo)

        #### RCE FOR PRESENT-DAY CO2
        atmosphere['CO2'][0] = 348/1e6


        T_sfc_1 = [1e8]
        toa_1 = [1e8]
        for i in range(nsteps):
            #surface fluxes
            z = height(atmosphere['plev'],atmosphere['T'][0],
                       atmosphere['phlev'][0],surface['temperature'])
            zini = 100

            T_ini,vmr_ini = T_vmr_z(T_atm_low,atmosphere['T'][0][0].copy(),
                                    atmosphere['H2O'][0,0].copy(),atmosphere['H2O'][0,1].copy(),
                                    z,zini)

            #compute the fluxes or input the forcing
            if coupled == True:
                LH = latent_heat(vmr_ini, T_ini,atmosphere['phlev'][0],RLS)
                SH = sensible_heat(T_ini,surface['temperature'],atmosphere['phlev'][0],RLS)

            else:
                LH = LH_f
                SH = SH_f


            #convective adjustment of the armosphere
            atmosphere,surface,radiation,net_rad_surface,atm_rad,\
            T_atm_low,E_imbalance,prec_mass,prec_heating,\
            RH,cold_point,prec_eff = RCPE_step_DSE(timestep,
                                                  atmosphere,surface,radiation,clearsky,
                                                  SH,LH,A,T_atm_low,
                                                  strong_coupling = strong,
                                                  constrain_RH = True,
                                                Flux_case = case)



            #update surface temperature with energetic fluxes (latent and radiative)
            if coupled == True:
                surface['temperature'] = surface['temperature'] + ((net_rad_surface - SH - LH)
                                       * seconds_day * timestep / surface.heat_capacity)

            else:
                surface['temperature'] = Ts_f



            status += 1
            if printcounter == 200:
                print(status/2)
            printcounter += 1

            T_sfc_1.append(surface['temperature'])
            toa_1.append(radiation['toa'].copy())
            if abs(T_sfc_1[i]-T_sfc_1[i-1])<tol and abs(toa_1[i]-toa_1[i-1])<tol:
                print(i)
                break

        print('hehe')
        results_file.write(str(Ts_f)+" "+str(case)+" "+str(coupled)
                           +" "+str(surface['temperature'])+" "+str(T_ini)
                           +" "+str(net_rad_surface)+" "+str(atm_rad)+" "+str(radiation['toa'])
                           +" "+str(E_imbalance)
                           +" "+str(LH)+" "+str(SH)+" "+str(prec_eff)
                           +" "+str(prec_mass)+" "+str(LH/Lv * seconds_day)+" "+str(RH[0])
                           +"\n")

results_file.close()

[285.]
[285.] 0.2
[285.] 0.2
100.5
14537
hehe
CPU times: user 5min 28s, sys: 999 ms, total: 5min 29s
Wall time: 5min 29s


In [5]:
print(radiation['sw_flxd'][0,-1])

356.42842363901576
